In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
from skimage.feature import local_binary_pattern
import joblib

def split_image(image, patch_size=400, overlap=0.5):
    height, width = image.shape[:2]
    step = int(patch_size * (1 - overlap))
    patches, coordinates = [], []
    for y in range(0, height - patch_size + 1, step):
        for x in range(0, width - patch_size + 1, step):
            patches.append(image[y:y + patch_size, x:x + patch_size])
            coordinates.append((x, y, x + patch_size, y + patch_size))
    return patches, coordinates

def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (128, 128))
    lbp = local_binary_pattern(resized, 8, 1, method='uniform')
    hist_lbp, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    hist_lbp = hist_lbp.astype("float")
    hist_lbp /= (hist_lbp.sum() + 1e-7)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], None, [8], [0, 180])
    hist_s = cv2.calcHist([hsv], [1], None, [8], [0, 256])
    hist_v = cv2.calcHist([hsv], [2], None, [8], [0, 256])
    hist_color = np.concatenate((hist_h, hist_s, hist_v)).flatten()
    hist_color = normalize(hist_color.reshape(1, -1))[0]

    return np.concatenate([hist_lbp, hist_color])

def train_svm(reference_folder):
    features, labels = [], []
    for label in ['positive', 'negative']:
        folder = os.path.join(reference_folder, label)
        for file in os.listdir(folder):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = cv2.imread(os.path.join(folder, file))
                if img is not None:
                    features.append(extract_features(img))
                    labels.append(1 if label == 'positive' else 0)
    clf = SVC(kernel='linear', probability=True)
    clf.fit(features, labels)
    joblib.dump(clf, 'arecanut_svm_model.pkl')
    return clf

def detect_arecanut(image, clf, patch_size=400, overlap=0.5, threshold=0.5):
    patches, coords = split_image(image, patch_size, overlap)
    boxes = []
    for patch, coord in zip(patches, coords):
        feat = extract_features(patch)
        prob = clf.predict_proba([feat])[0][1]
        if prob >= threshold:
            boxes.append(coord)
    return boxes

def compute_iou(box1, box2):
    x1, y1 = max(box1[0], box2[0]), max(box1[1], box2[1])
    x2, y2 = min(box1[2], box2[2]), min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return inter_area / (area1 + area2 - inter_area + 1e-7)

def merge_boxes(boxes, iou_threshold=0.3):
    merged = []
    while boxes:
        base = boxes.pop(0)
        overlapping = [b for b in boxes if compute_iou(base, b) > iou_threshold]
        for b in overlapping:
            boxes.remove(b)
        all_boxes = [base] + overlapping
        x1 = min(b[0] for b in all_boxes)
        y1 = min(b[1] for b in all_boxes)
        x2 = max(b[2] for b in all_boxes)
        y2 = max(b[3] for b in all_boxes)
        merged.append((x1, y1, x2, y2))
    return merged

def save_results(image, boxes, image_name, output_image_folder, output_label_folder):
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    for box in boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imwrite(os.path.join(output_image_folder, image_name), image)

    h, w = image.shape[:2]
    label_path = os.path.join(output_label_folder, os.path.splitext(image_name)[0] + '.txt')
    with open(label_path, 'w') as f:
        for x1, y1, x2, y2 in boxes:
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            bw = (x2 - x1) / w
            bh = (y2 - y1) / h
            f.write(f"0 {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n")




In [ ]:
if __name__ == "__main__":
    # Train once and save model
    reference_images = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/reference_images"
    clf = train_svm(reference_images)  # contains 'positive' and 'negative' subfolders

    # Input and output paths
    input_image_folder = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/test_images"  # your input folder path
    output_image_folder = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/using_svm/output_images"
    output_label_folder = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/using_svm/output_labels"

    # Create output directories if they don't exist
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    # Process each image in the input folder
    for img_name in os.listdir(input_image_folder):
        img_path = os.path.join(input_image_folder, img_name)

        # Skip if not an image file
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        # Load image and detect arecanut
        img = cv2.imread(img_path)
        if img is None:
            print(f"Could not read image: {img_path}")
            continue

        boxes = detect_arecanut(img, clf)
        merged_boxes = merge_boxes(boxes)

        # Save bounding box results and labels
        save_results(img, merged_boxes, img_name, output_image_folder, output_label_folder)
        print(f"Processed image: {img_name}")

        print(f"Successfully processed {img_name}")

Processed image: image3.jpg
Successfully processed image3.jpg
Processed image: image1.jpg
Successfully processed image1.jpg
Processed image: image2.jpg
Successfully processed image2.jpg
Processed image: image4.jpg
Successfully processed image4.jpg
Processed image: image5.jpg
Successfully processed image5.jpg
Processed image: image6.jpg
Successfully processed image6.jpg
Processed image: image7.jpg
Successfully processed image7.jpg
Processed image: image8.jpg
Successfully processed image8.jpg
Processed image: image-255.jpg
Successfully processed image-255.jpg
Processed image: image-108.jpg
Successfully processed image-108.jpg
Processed image: image-1.jpg
Successfully processed image-1.jpg
Processed image: image-103.jpg
Successfully processed image-103.jpg
Processed image: image-178.jpg
Successfully processed image-178.jpg
Processed image: image-71.jpg
Successfully processed image-71.jpg
Processed image: image-131.jpg
Successfully processed image-131.jpg
Processed image: image-322.jpg
Suc

In [ ]:
import cv2
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import normalize
from skimage.feature import local_binary_pattern
import joblib

def split_image(image, patch_size=400, overlap=0.5):
    height, width = image.shape[:2]
    step = int(patch_size * (1 - overlap))
    patches, coordinates = [], []
    for y in range(0, height - patch_size + 1, step):
        for x in range(0, width - patch_size + 1, step):
            patches.append(image[y:y + patch_size, x:x + patch_size])
            coordinates.append((x, y, x + patch_size, y + patch_size))
    return patches, coordinates

def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (128, 128))
    lbp = local_binary_pattern(resized, 8, 1, method='uniform')
    hist_lbp, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    hist_lbp = hist_lbp.astype("float")
    hist_lbp /= (hist_lbp.sum() + 1e-7)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], None, [8], [0, 180])
    hist_s = cv2.calcHist([hsv], [1], None, [8], [0, 256])
    hist_v = cv2.calcHist([hsv], [2], None, [8], [0, 256])
    hist_color = np.concatenate((hist_h, hist_s, hist_v)).flatten()
    hist_color = normalize(hist_color.reshape(1, -1))[0]

    return np.concatenate([hist_lbp, hist_color])

def train_rf(reference_folder):
    features, labels = [], []
    for label in ['positive', 'negative']:
        folder = os.path.join(reference_folder, label)
        for file in os.listdir(folder):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img = cv2.imread(os.path.join(folder, file))
                if img is not None:
                    features.append(extract_features(img))
                    labels.append(1 if label == 'positive' else 0)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(features, labels)
    joblib.dump(clf, 'arecanut_rf_model.pkl')
    return clf

def detect_arecanut(image, clf, patch_size=400, overlap=0.5, threshold=0.5):
    patches, coords = split_image(image, patch_size, overlap)
    boxes = []
    for patch, coord in zip(patches, coords):
        feat = extract_features(patch)
        prob = clf.predict_proba([feat])[0][1]
        if prob >= threshold:
            boxes.append(coord)
    return boxes

def compute_iou(box1, box2):
    x1, y1 = max(box1[0], box2[0]), max(box1[1], box2[1])
    x2, y2 = min(box1[2], box2[2]), min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return inter_area / (area1 + area2 - inter_area + 1e-7)

def merge_boxes(boxes, iou_threshold=0.3):
    merged = []
    while boxes:
        base = boxes.pop(0)
        overlapping = [b for b in boxes if compute_iou(base, b) > iou_threshold]
        for b in overlapping:
            boxes.remove(b)
        all_boxes = [base] + overlapping
        x1 = min(b[0] for b in all_boxes)
        y1 = min(b[1] for b in all_boxes)
        x2 = max(b[2] for b in all_boxes)
        y2 = max(b[3] for b in all_boxes)
        merged.append((x1, y1, x2, y2))
    return merged

def save_results(image, boxes, image_name, output_image_folder, output_label_folder):
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    for box in boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imwrite(os.path.join(output_image_folder, image_name), image)

    h, w = image.shape[:2]
    label_path = os.path.join(output_label_folder, os.path.splitext(image_name)[0] + '.txt')
    with open(label_path, 'w') as f:
        for x1, y1, x2, y2 in boxes:
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            bw = (x2 - x1) / w
            bh = (y2 - y1) / h
            f.write(f"0 {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n")


In [ ]:
if __name__ == "__main__":
    # Train once and save model
    reference_images = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/reference_images"
    clf = train_svm(reference_images)  # contains 'positive' and 'negative' subfolders

    # Input and output paths
    input_image_folder = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/test_images"
    output_image_folder = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/using_random_forest/output_images"
    output_label_folder = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/using_random_forest/output_labels"

    # Create output directories if they don't exist
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    # Process each image in the input folder
    for img_name in os.listdir(input_image_folder):
        img_path = os.path.join(input_image_folder, img_name)

        # Skip if not an image file
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        # Load image and detect arecanut
        try:
            img = cv2.imread(img_path)
            if img is None:
                print(f"Could not read image: {img_path}")
                continue

            boxes = detect_arecanut(img, clf)
            merged_boxes = merge_boxes(boxes)

            # Save bounding box results and labels
            save_results(img, merged_boxes, img_name, output_image_folder, output_label_folder)
            print(f"Processed image: {img_name}")

            print(f"Successfully processed {img_name}")

        except Exception as e:
            print(f"Error processing {img_name}: {str(e)}")

Processed image: image3.jpg
Successfully processed image3.jpg
Processed image: image1.jpg
Successfully processed image1.jpg
Processed image: image2.jpg
Successfully processed image2.jpg
Processed image: image4.jpg
Successfully processed image4.jpg
Processed image: image5.jpg
Successfully processed image5.jpg
Processed image: image6.jpg
Successfully processed image6.jpg
Processed image: image7.jpg
Successfully processed image7.jpg
Processed image: image8.jpg
Successfully processed image8.jpg
Processed image: image-255.jpg
Successfully processed image-255.jpg
Processed image: image-108.jpg
Successfully processed image-108.jpg
Processed image: image-1.jpg
Successfully processed image-1.jpg
Processed image: image-103.jpg
Successfully processed image-103.jpg
Processed image: image-178.jpg
Successfully processed image-178.jpg
Processed image: image-71.jpg
Successfully processed image-71.jpg
Processed image: image-131.jpg
Successfully processed image-131.jpg
Processed image: image-322.jpg
Suc

In [ ]:
import cv2
import os
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from skimage.feature import local_binary_pattern
import joblib
from collections import defaultdict

# Enhanced feature extraction
def extract_features(image):
    # Color features
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], None, [16], [0, 180])
    hist_s = cv2.calcHist([hsv], [1], None, [16], [0, 256])
    hist_v = cv2.calcHist([hsv], [2], None, [16], [0, 256])

    # Texture features
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (128, 128))
    lbp = local_binary_pattern(resized, 24, 3, method='uniform')
    hist_lbp, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))

    # Edge features
    edges = cv2.Canny(gray, 100, 200)
    edge_density = np.sum(edges) / (edges.shape[0] * edges.shape[1])

    # Combine all features
    features = np.concatenate([
        hist_h.flatten(),
        hist_s.flatten(),
        hist_v.flatten(),
        hist_lbp.flatten(),
        [edge_density]
    ])

    return features

def split_image(image, patch_size=400, overlap=0.5):
    height, width = image.shape[:2]
    step = int(patch_size * (1 - overlap))
    patches, coordinates = [], []
    for y in range(0, height - patch_size + 1, step):
        for x in range(0, width - patch_size + 1, step):
            patches.append(image[y:y + patch_size, x:x + patch_size])
            coordinates.append((x, y, x + patch_size, y + patch_size))
    return patches, coordinates

# Improved training with standardization
def train_logistic_regression(reference_folder):
    features, labels = [], []

    for label in ['positive', 'negative']:
        folder = os.path.join(reference_folder, label)
        for file in os.listdir(folder):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(folder, file)
                img = cv2.imread(img_path)
                if img is not None:
                    features.append(extract_features(img))
                    labels.append(1 if label == 'positive' else 0)

    # Standardize features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    # Train with class weights to handle imbalance
    model = LogisticRegression(max_iter=1000, class_weight='balanced', C=0.1)
    model.fit(features, labels)

    # Save both model and scaler
    joblib.dump({'model': model, 'scaler': scaler}, 'arecanut_logistic_model.pkl')
    return model, scaler

# Advanced box processing
def process_boxes(boxes, min_area=5000, overlap_thresh=0.3):
    if not boxes:
        return []

    # Filter small boxes
    boxes = [box for box in boxes if (box[2]-box[0])*(box[3]-box[1]) > min_area]

    # Remove nested boxes
    to_remove = set()
    for i, box1 in enumerate(boxes):
        for j, box2 in enumerate(boxes):
            if i != j and box1[0] >= box2[0] and box1[1] >= box2[1] and \
               box1[2] <= box2[2] and box1[3] <= box2[3]:
                to_remove.add(i)

    boxes = [box for i, box in enumerate(boxes) if i not in to_remove]

    # Convert to numpy array for processing
    boxes = np.array(boxes)
    if len(boxes) == 0:
        return []

    # Non-maximum suppression
    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))

    return boxes[pick].tolist()

# Enhanced detection with confidence
def detect_arecanut(image, model, scaler, patch_size=400, overlap=0.5, confidence_thresh=0.7):
    patches, coords = split_image(image, patch_size, overlap)
    boxes = []
    confidences = []

    for patch, coord in zip(patches, coords):
        feat = extract_features(patch)
        feat = scaler.transform([feat])
        prob = model.predict_proba(feat)[0][1]

        if prob >= confidence_thresh:
            boxes.append(coord)
            confidences.append(prob)

    # Filter based on confidence
    if len(confidences) > 0:
        median_conf = np.median(confidences)
        boxes = [box for box, conf in zip(boxes, confidences) if conf >= median_conf]

    return process_boxes(boxes)

def save_results(image, boxes, image_name, output_image_folder, output_label_folder):
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    # Draw boxes on image
    output_img = image.copy()
    for box in boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(output_img, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Save image with boxes
    cv2.imwrite(os.path.join(output_image_folder, image_name), output_img)

    # Save labels in YOLO format
    h, w = image.shape[:2]
    label_path = os.path.join(output_label_folder, os.path.splitext(image_name)[0] + '.txt')
    with open(label_path, 'w') as f:
        for box in boxes:
            x_center = ((box[0] + box[2]) / 2) / w
            y_center = ((box[1] + box[3]) / 2) / h
            width = (box[2] - box[0]) / w
            height = (box[3] - box[1]) / h
            f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")



In [ ]:
if __name__ == "__main__":
    # Model paths
    model_path = 'arecanut_logistic_model.pkl'
    reference_path = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/reference_images"

    # Train or load model
    if not os.path.exists(model_path):
        print("Training new model...")
        model, scaler = train_logistic_regression(reference_path)
    else:
        print("Loading pre-trained model...")
        try:
            saved_data = joblib.load(model_path)
            model = saved_data['model']
            scaler = saved_data['scaler']
        except:
            print("Model format mismatch, training new model...")
            model, scaler = train_logistic_regression(reference_path)

    # Process images
    input_folder = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/test_images"
    output_img_folder = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/using_logistic_regression/output_images"
    output_label_folder = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/Model_Statistical_Detection/using_logistic_regression/output_labels"

    os.makedirs(output_img_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    for img_name in sorted(os.listdir(input_folder)):
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        img_path = os.path.join(input_folder, img_name)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Could not read image: {img_name}")
            continue

        print(f"Processing {img_name}...")

        try:
            boxes = detect_arecanut(img, model, scaler)
            save_results(
                image=img,
                boxes=boxes,
                image_name=img_name,
                output_image_folder=output_img_folder,
                output_label_folder=output_label_folder
            )
            print(f"Found {len(boxes)} plantations in {img_name}")

        except Exception as e:
            print(f"Error processing {img_name}: {str(e)}")

Training new model...
Processing image-1.jpg...
Found 1 plantations in image-1.jpg
Processing image-100.jpg...
Found 1 plantations in image-100.jpg
Processing image-101.jpg...
Found 0 plantations in image-101.jpg
Processing image-102.jpg...
Found 0 plantations in image-102.jpg
Processing image-103.jpg...
Found 2 plantations in image-103.jpg
Processing image-108.jpg...
Found 2 plantations in image-108.jpg
Processing image-11.jpg...
Found 0 plantations in image-11.jpg
Processing image-112.jpg...
Found 2 plantations in image-112.jpg
Processing image-116.jpg...
Found 2 plantations in image-116.jpg
Processing image-117.jpg...
Found 2 plantations in image-117.jpg
Processing image-119.jpg...
Found 2 plantations in image-119.jpg
Processing image-122.jpg...
Found 1 plantations in image-122.jpg
Processing image-125.jpg...
Found 2 plantations in image-125.jpg
Processing image-126.jpg...
Found 2 plantations in image-126.jpg
Processing image-127.jpg...
Found 2 plantations in image-127.jpg
Processin